In [3]:
import pandas as pd
import os
import numpy as np
import geopandas as gpd
import folium

## Organizing coordinates

In [2]:
os.chdir('data')

In [3]:
os.listdir()
coords_fn = 'Koordinates+Elevation.xlsx'
coords_df = pd.read_excel(coords_fn)
coords_df = coords_df.iloc[:,:-1]

col_names_list = ['WellID', 'CaseHeight', 'E', 'N', 'Type']
names_dict = {}
for i in range(len(col_names_list)):
    names_dict [coords_df.iloc[:,i].name] = col_names_list[i]

coords_df = coords_df.rename(columns = names_dict)
coords_df.head()

,WellID,CaseHeight,E,N,Type
0,G1,119.49,5424455.69,5648391.10,NaN
1,G2,118.83,5424474.00,5648390.00,no existing well
2,G3,118.73,5424491.00,5648392.00,no existing well
3,G4,119.19,5424509.75,5648392.55,NaN
4,G4 neu,120.09,NaN,NaN,NaN


## Coordinates completion

After organizing the table above mentioned, we went to the field and collected coordinates manually. Some of them were computed directly in the field by measuring the distance from the well with missing coordinates to a well with known coordinates. Some others had no well closeby, therefore the group measured the distance to the closest corner of the site or of the building and saved it to compute the coordinates later on. An *e-mail was sent to Zhao* asking for a georrefenced file. The group intends to use this file to extract the coordinates of the known points and compute the missing well coordinates.

Thomas said by e-mail on the 28-10-2022 that the coordinate system of the area is Gauss-Krüger.

Now below, I compute the partial coordinates and convert it from Gauss-Krüger UTM tto WGS 1984.

According to [epsg](https://www.geoportal.rlp.de/mediawiki/index.php/EPSG-Codes/de), the epsg of the coordinate system is 31469. We convert from UTM to GCS WGS 1984, epsg 4326. It can also be manually converted from [here](https://www.koordinaten-umrechner.de/decimal/51.000000,10.000000?karte=OpenStreetMap&zoom=8).


In [4]:
os.chdir('D:/Repos/PirnaCaseStudy/Data/Tables')
os.listdir()

['database.db',
 'Divers.csv',
 'DrillingTests.csv',
 'MonitoringPoints.csv',
 'Old',
 'Points.csv',
 'PointsMeasurements2.csv',
 'PointsMeasurements3.csv',
 'TestsType.csv',
 'Variables.csv',
 'WellDiver.csv']

Read data

In [3]:
#define output epsg
epsg = 4326 #WGS 1984

Points_df = pd.read_csv('Points.csv')

Points_df = Points_df[(~Points_df.E.isna() )].reset_index(drop=1)


#generating points
Points_df['geometry'] = gpd.points_from_xy(Points_df.E, Points_df.N, crs = 'EPSG:{}'.format(epsg) )
Points_df = Points_df.drop(columns = ['E', 'N'])

Points_gdf = gpd.GeoDataFrame(Points_df, crs = 'EPSG:{}'.format(epsg)) #generate gdf
Points_gdf.tail()

,ID,Name,Type,DescriptionData,MonitoringPoint,DrillingTest,Depth,Unnamed: 9,geometry
29,29,D-GWM5,Drill,1,1,0.0,14.0,NaN,POINT (13.92378 50.96581)
30,30,D-GWM6,Drill,1,1,0.0,14.0,NaN,POINT (13.92383 50.96583)
31,31,D-GWM7,Drill,1,1,0.0,14.0,NaN,POINT (13.92391 50.96585)
32,32,D-P12,Drill,0,1,0.0,14.8,NaN,POINT (13.92387 50.96584)
33,33,C-RG,River Gage,1,1,NaN,NaN,NaN,POINT (13.92975 50.96459)


Transform it to geodataframe convert coordinates, and plot it.

# Plotting

In [4]:
y1 = Points_gdf [Points_gdf.Name == 'D-G10'].geometry.y.values[0]

In [6]:
ControlPoint = Points_gdf [Points_gdf.Name == 'D-G17'].geometry
y2 = ControlPoint.y.values[0]
dy = y1-y2 + 4e-4

ControlPointy = y2 - dy
ControlPointx = ControlPoint.x.values[0] - 2e-3
ControlPoint = [ControlPointy, ControlPointx]

In [165]:
ControlPoint

[50.96454391, 13.92163433]

In [8]:
def ControlPoints(n=18):
    '''
    Function to distribute fictious points in the river border and interpolate the levels
    Use n = 18 gives 18 points back. That is a perfect number.
    '''
    ControlPoint = [50.96454391, 13.92163433] #fictious point close to the site
    river_angle = 9.8 #angles with the east in degrees
    river_anglerad = river_angle * 2 * np.pi / 360 #angle in rad
    hypotenuse = 3e-4 #distance between points
    dx = np.cos(river_anglerad) * hypotenuse
    dy = np.sin(river_anglerad) * hypotenuse
    x = np.arange(ControlPoint[1], ControlPoint[1] + (n)*dx , dx)
    y = np.arange(ControlPoint[0], ControlPoint[0] + (n)*dy , dy)
    control_points_list = [[y[i],x[i]] for i,j in enumerate(x)]
    df = pd.DataFrame(control_points_list, columns = ['y', 'x'])
    return df

In [14]:
# Create a geometry list from the GeoDataFrame
geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in Points_gdf.geometry]

#control points
control_points_df = ControlPoints()

map_center = Points_gdf.geometry.y.mean(), Points_gdf.geometry.x.mean() #in folium y comes before x

#create object map
map = folium.Map(location=map_center, tiles="OpenStreetMap", zoom_start=16)

i = 0
for coordinates in geo_df_list:
    # Place the markers with the popup labels and data'
    map.add_child(
        folium.Marker(
            location = coordinates,
            popup = "Point: " + Points_gdf.Name[i]
        )
    )
    i += 1

i = 0
for coordinates in control_points_df.iterrows():
    # Place the markers with the popup labels and data'
    map.add_child(
        folium.Marker(
            location = (control_points_df['y'][i], control_points_df['x'][i]),
            popup = f'Control Point {i}',
            icon=folium.Icon(color="%s" % 'red'),
        )
    )
    i+=1
map

## Manual editting and others...

In [9]:
from shapely.geometry import point

In [2]:
x = 424845
y = 5646432
src_crs = 'EPSG:25833'
dst_crs = 'EPSG:4326'

In [31]:
geometry = gpd.points_from_xy( [x], [y], crs = src_crs)
gdf = gpd.GeoDataFrame(geometry = geometry)
gdf = gdf.to_crs(dst_crs)
print(f'River Gage coordinates are: X: {gdf.geometry.x.values[0]} Y: {gdf.geometry.y.values[0]} in WGS 1984 EPSG 4326')


River Gage coordinates are: X: 13.929753816652829 Y: 50.964586171556476 in WGS 1984 EPSG 4326


In [32]:
LevelZero = 108.006